# Sandbox Notebook

Experimental notebook for testing `home_media` package functionality.

Auto-reload is enabled to pick up changes to the package without restarting the kernel.

In [ ]:
# Enable auto-reload for the home_media package
%load_ext autoreload
%autoreload 2

In [ ]:
# Imports
import sys
from pathlib import Path
import yaml
import pandas as pd
from datetime import datetime

# Add parent directory to path so we can import home_media
sys.path.insert(0, str(Path.cwd().parent))

# Import home_media package
from home_media import (
    scan_directory,
    group_files_to_images,
    extract_base_name,
    Image,
    ImageFile,
    FileRole,
    FileFormat,
)
from home_media.scanner import list_subdirectories

## Load Configuration

In [ ]:
# Load config from config.yaml
config_path = Path.cwd().parent / "config.yaml"

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

photos_root = Path(config['photos_root_original'])

print("Configuration loaded:")
print(f"photos_root_original: {photos_root}")

## List Subdirectories in photos_root_original

In [ ]:
# Use home_media to list subdirectories
subdirs = list_subdirectories(photos_root)

print(f"Found {len(subdirs)} subdirectories in: {photos_root}\n")

for subdir in subdirs[:20]:  # Show first 20
    print(f"  📁 {subdir.name}")

if len(subdirs) > 20:
    print(f"\n  ... and {len(subdirs) - 20} more")

## Scan a Specific Subdirectory

Let's examine a specific subdirectory to see what images and files we have.

In [ ]:
# Choose a subdirectory to examine
subdir = "2025/01/01"
target_dir = photos_root / subdir

print(f"Scanning directory: {target_dir}")
print(f"Relative path: {subdir}")
print()

# Use home_media to scan and group files
images_df, files_df = scan_directory(target_dir, photos_root=photos_root)

print(f"Found {len(images_df)} images with {len(files_df)} total files")

## Images DataFrame

One row per image (moment in time), with aggregated information about all its files.

In [ ]:
# Display images DataFrame
print("Images DataFrame:")
print(f"Columns: {list(images_df.columns)}\n")

# Show first 10 images
images_df.head(10)

In [ ]:
# Summary statistics
print("Image Statistics:\n")
print(f"Total images: {len(images_df)}")
print(f"Images with RAW files: {images_df['has_raw'].sum()}")
print(f"Images with JPEG files: {images_df['has_jpeg'].sum()}")
print(f"Images with sidecars: {images_df['has_sidecar'].sum()}")
print(f"\nAverage files per image: {images_df['file_count'].mean():.1f}")
print(f"Max files per image: {images_df['file_count'].max()}")
print(f"\nTotal storage: {images_df['total_size_bytes'].sum() / (1024**3):.2f} GB")

## Files DataFrame

One row per file, linked to images by `base_name` and `subdirectory`.

In [ ]:
# Display files DataFrame
print("Files DataFrame:")
print(f"Columns: {list(files_df.columns)}\n")

# Show first 10 files
files_df.head(10)

In [ ]:
# File format breakdown
print("File Format Distribution:\n")
format_counts = files_df['format'].value_counts()
for format_type, count in format_counts.items():
    print(f"  {format_type}: {count}")

print("\nFile Role Distribution:\n")
role_counts = files_df['role'].value_counts()
for role, count in role_counts.items():
    print(f"  {role}: {count}")

## Working with Image Objects

For more complex operations, we can work with the `Image` objects directly.

In [ ]:
# Get file paths and group into Image objects
file_paths = [f for f in target_dir.iterdir() if f.is_file()]
images = group_files_to_images(file_paths, photos_root=photos_root)

print(f"Created {len(images)} Image objects\n")

# Examine the first image in detail
if images:
    img = images[0]
    print(f"Image: {img.base_name}")
    print(f"  Subdirectory: {img.subdirectory}")
    print(f"  File count: {img.file_count}")
    print(f"  Has RAW: {img.has_raw}")
    print(f"  Has JPEG: {img.has_jpeg}")
    print(f"  Has sidecar: {img.has_sidecar}")
    print(f"  Total size: {img.total_size_bytes:,} bytes")
    print(f"  Suffixes: {img.suffixes}")
    print(f"\n  Files:")
    for f in img.files:
        print(f"    - {f.filename}")
        print(f"      Role: {f.role.name}, Format: {f.format.value}, Size: {f.file_size_bytes:,} bytes")

## Filename Pattern Analysis

Let's see how the `extract_base_name()` function handles different filename patterns.

In [ ]:
# Test different filename patterns
test_filenames = [
    "2025-01-01_00-28-40.jpg",
    "2025-01-01_00-28-40.CR3",
    "2025-01-01_00-28-40_001.jpg",
    "2025-01-01_00-28-40.jpg.xmp",
    "PXL_20251210_200246684.RAW-01.COVER.jpg",
    "PXL_20251210_200246684.RAW-02.ORIGINAL.dng",
    "photo_edit.jpg",
]

print("Filename Pattern Analysis:\n")
for filename in test_filenames:
    base_name, suffix = extract_base_name(filename)
    print(f"{filename}")
    print(f"  base_name: {base_name}")
    print(f"  suffix: {suffix}")
    print()

## Images with Multiple Files

Find images that have multiple files (RAW+JPEG pairs, sidecars, etc.).

In [ ]:
# Filter images with multiple files
multi_file_images = images_df[images_df['file_count'] > 1]

print(f"Images with multiple files: {len(multi_file_images)} of {len(images_df)}\n")

# Show examples
print("Examples:")
for idx, row in multi_file_images.head(5).iterrows():
    print(f"\n{row['base_name']}:")
    print(f"  Files: {row['file_count']}")
    print(f"  Suffixes: {row['suffixes']}")
    print(f"  Has RAW: {row['has_raw']}, Has JPEG: {row['has_jpeg']}, Has sidecar: {row['has_sidecar']}")

## Canonical Naming Preview

Preview how files would be renamed using the canonical naming scheme: `YYYY/mm/dd/YYYY-mm-dd_HH-MM-SS` + suffix.

In the future, the capture time will be extracted from EXIF data.

In [ ]:
# Preview canonical naming for the first image
if images:
    img = images[0]

    # Simulate EXIF capture time (in real use, this comes from EXIF)
    # Try to parse from the base_name if it follows our standard format
    try:
        # Try to parse datetime from base_name like "2025-01-01_00-28-40"
        if len(img.base_name) >= 19 and img.base_name[10] == '_':
            simulated_capture_time = datetime.strptime(img.base_name[:19], "%Y-%m-%d_%H-%M-%S")
        else:
            simulated_capture_time = datetime(2025, 1, 1, 0, 28, 40)
    except Exception:
        simulated_capture_time = datetime(2025, 1, 1, 0, 28, 40)

    print("Current naming:")
    print(f"  Base name: {img.base_name}")
    print(f"  Subdirectory: {img.subdirectory}")
    print(f"  Files: {img.file_count}")

    print(f"\nCanonical naming (based on capture time: {simulated_capture_time}):")
    print(f"  Base name: {img.get_canonical_name(simulated_capture_time)}")
    print(f"  Subdirectory: {img.get_canonical_subdirectory(simulated_capture_time)}")

    print(f"\nFull canonical paths would be:")
    for f in img.files:
        canonical_path = (
            f"{img.get_canonical_subdirectory(simulated_capture_time)}/"
            f"{img.get_canonical_name(simulated_capture_time)}{f.suffix}"
        )
        print(f"  {canonical_path}")

## Query Examples

Use pandas to query the data in various ways.

In [ ]:
# Find images that only have RAW files (no JPEG)
raw_only = images_df[(images_df['has_raw']) & (~images_df['has_jpeg'])]
print(f"Images with only RAW (no JPEG): {len(raw_only)}")
raw_only.head()

In [ ]:
# Find images with XMP sidecars
with_sidecar = images_df[images_df['has_sidecar']]
print(f"Images with XMP sidecars: {len(with_sidecar)}")
with_sidecar.head()

In [ ]:
# Find all CR3 (Canon RAW) files
cr3_files = files_df[files_df['format'] == 'cr3']
print(f"Canon RAW (CR3) files: {len(cr3_files)}")
cr3_files.head()

In [ ]:
# Largest images by total size
largest_images = images_df.nlargest(10, 'total_size_bytes')[['base_name', 'file_count', 'total_size_bytes']]
largest_images['size_mb'] = largest_images['total_size_bytes'] / (1024**2)
print("Largest images by total size:\n")
largest_images